In [7]:
import sys
sys.path.append("../utils")
from utils import load_data
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr
# Load the data
data_train, data_test , targets_train= load_data(raw=False)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data_train, targets_train, test_size=0.2, random_state=42)

# Train a Random Forest model
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)

# Predict on validation data
y_val_pred = random_forest.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
spearman_non_encoded, p_value_non_encoded = spearmanr(y_val, y_val_pred)

print("Validation Mean Squared Error (MSE):", mse)
print("Validation R² Score:", r2)
print("Spearman's Correlation (ρ):", spearman_non_encoded)

/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Validation Mean Squared Error (MSE): 0.00565469179517595
Validation R² Score: 0.24463911972268548
Spearman's Correlation (ρ): 0.41872883976875536


In [12]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score

import sys
sys.path.append("../utils")
from utils import load_data

# Load the data
data_train, data_test , targets_train= load_data(raw=False)

# Split the data into training and validation sets
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data_train, targets_train, test_size=0.2, random_state=42)

# Define the parameter grid with more options for a thorough search
param_grid = {
    'n_estimators': [100, 200, 300, 400, 500],  # Wider range for number of trees
    'max_depth': [5, 10, 15, 20, 30, None],     # Include None for unrestricted depth
    'min_samples_split': [2, 5, 10],            # Minimum samples for a split
    'min_samples_leaf': [1, 2, 4, 8],           # Leaf node regularization
    'max_features': ['sqrt', 'log2']            # Avoid 'auto' (deprecated in sklearn)
}

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_grid=param_grid,
    cv=5,                              # Increase cross-validation folds
    scoring='neg_mean_squared_error',  # Try alternative scoring metrics
    verbose=2,
    n_jobs=-1
)

# Perform grid search
grid_search.fit(X_train, y_train)

# Extract best parameters and evaluate
best_model = grid_search.best_estimator_

# Validation set evaluation
y_val_pred = best_model.predict(X_val)
mse = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
spearman_non_encoded, p_value_non_encoded = spearmanr(y_val, y_val_pred)

print("Validation Mean Squared Error (MSE):", mse)
print("Validation R² Score:", r2)
print("Spearman's Correlation (ρ):", spearman_non_encoded)


Fitting 5 folds for each of 720 candidates, totalling 3600 fits


/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473:

[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.5s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.5s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.6s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.6s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   2.8s


/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473:

[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.3s
[CV] END max_depth=5, max_features=sqrt, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   4.2s


...

/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=  14.7s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=  14.8s
[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=400; total time=  14.1s


/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV] END max_depth=30, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   3.2s


/Users/lilameflah/anaconda3/envs/comsocsci2024/lib/python3.11/site-packages/sklearn/base.py:1473: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

import sys
sys.path.append("../utils")
from utils import load_data

# Load the data
data_train, data_test , targets_train= load_data(raw= False)


# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an initial Random Forest model to get feature importances
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Get feature importances and select the top 500 features
importances = rf.feature_importances_
important_indices = np.argsort(importances)[::-1][:500]  # Top 500 important features
X_train_reduced = X_train.iloc[:, important_indices]
X_val_reduced = X_val.iloc[:, important_indices]

# Train a new Random Forest model on the reduced feature set
rf_reduced = RandomForestRegressor(n_estimators=300, random_state=42)
rf_reduced.fit(X_train_reduced, y_train)

# Predict and evaluate
y_val_pred = rf_reduced.predict(X_val_reduced)
mse = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
spearman_non_encoded, p_value_non_encoded = spearmanr(y_val, y_val_pred)

print("Spearman's Correlation (ρ):", spearman_non_encoded)
print("Validation Mean Squared Error (MSE):", mse)
print("Validation R² Score:", r2)

Spearman's Correlation (ρ): 0.38886848255818873
Validation Mean Squared Error (MSE): 0.005764102991137482
Validation R² Score: 0.2300238338172358


In [9]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import sys
sys.path.append("../utils")
from utils import load_data

# Load the data
data_train, data_test , targets_train= load_data(True)

# Preprocess the data
# Drop the "Unnamed: 0" column (cell line identifiers) as it's not a feature
X = data_train.drop(columns=["Unnamed: 0"])
y = targets_train["AAC"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an XGBoost model
xgb_model = XGBRegressor(
    n_estimators=100,       # Number of trees
    learning_rate=0.05,     # Learning rate
    max_depth=6,            # Maximum depth of each tree
    random_state=42,
    n_jobs=-1               # Use all available CPU cores
)
xgb_model.fit(X_train, y_train)

# Predict on validation data
y_val_pred = xgb_model.predict(X_val)

# Evaluate the model
mse = mean_squared_error(y_val, y_val_pred)
r2 = r2_score(y_val, y_val_pred)
spearman_non_encoded, p_value_non_encoded = spearmanr(y_val, y_val_pred)

print("Validation Mean Squared Error (MSE):", mse)
print("Validation R² Score:", r2)
print("Spearman's Correlation (ρ):", spearman_non_encoded)





KeyboardInterrupt: 

In [9]:
#use MLP to predict the AAC
import pandas as pd
import numpy as np
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import sys

sys.path.append("../utils")

from utils import load_data

# Load the data
data_train, data_test , targets_train= load_data(True)

# Preprocess the data
# Drop the "Unnamed: 0" column (cell line identifiers) as it's not a feature
X = data_train.drop(columns=["Unnamed: 0"])
y = targets_train["AAC"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train an MLP model
mlp_model = MLPRegressor(
    hidden_layer_sizes=(100, 50),  # Two hidden layers with 100 and 50 neurons
    activation='relu',
    solver='adam',
    learning_rate='adaptive',
    max_iter=1000,
    random_state=42
)
mlp_model.fit(X_train, y_train)

# Predict on validation data

y_val_pred = mlp_model.predict(X_val)

# Evaluate the model

mse = mean_squared_error(y_val, y_val_pred)

r2 = r2_score(y_val, y_val_pred)

print("Validation Mean Squared Error (MSE):", mse)


c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_typing\_scalars.py:12: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  _BoolLike_co = Union[bool, np.bool]


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\rache\AppData\Local\Temp\ipykernel_56196\4172490506.py", line 12, in <module>
    from utils import load_data
  File "c:\Users\Bureau\ml-project-2-lol-ml\src\notebooks\../utils\utils.py", line 7, in <module>
    import tensorflow as tf
  File "c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\__init__.py", line 49, in <module>
    from tensorflow._api.v2 import __internal__
  File "c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\_api\v2\__internal__\__init__.py", line 8, in <module>
    from tensorflow._api.v2.__internal__ import autograph
  File "c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\tensorflow\_api\v2\__internal__\autograph\__init__.py", li